In [1]:
import pandas as pd
from smart_open import open
import boto3
from tqdm import tqdm
import numpy as np

# Import credentials
import credentials

In [2]:
# Create boto3 session
session = boto3.session.Session()

# Transport params for smart_open
transport_params = {
    "client": session.client(
        "s3",
        endpoint_url=credentials.MINIO_URL,
        aws_access_key_id=credentials.ACCESS_KEY,
        aws_secret_access_key=credentials.SECRET_KEY
    )
}